In [5]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['번역(한-영)', '번역(영-한)', '영어 번역 교정', '한국어 번역 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            source_language = line['sourceLanguage']
            source_text = line['segments']['sourceText']
            target_language = line['targetLanguage']
            target_text_mt = line['segments']['mtText']
            if 'targetText' not in line['segments']:
                continue
            target_text = line['segments']['targetText']
            
            #### data preprocess end 
            
            data = {'input': {}, 'output': None}
            data['input']['source_text'] = source_text
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            data['output'] = target_text
            
            
            #### 번역(한-영)
            if source_language == 'ko':
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-영) end
            
            
            #### 번역(영-한)
            if target_language == 'ko':
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(영-한) end
            
            if target_text != target_text_mt:
                data['output'] = [target_text, target_text_mt]
                
                #### 영어 번역 교정
                ### 'output' is list of output sorted by relevance (1st is most relevant)
                if target_language == 'en':
                    task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 영어 번역 교정 end
                
                
                #### 한국어 번역 교정
                ### 'output' is list of output sorted by relevance (1st is most relevant)
                if target_language == 'ko':
                    task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 한국어 번역 교정 end
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]
